<h1/> Reubens Assessment <h1>

This notebook will cover the capstone project in whole

In [33]:
print("Hello Capstone project Course!")

Hello Capstone project Course!


<h2> Week 3 Assessment

In [28]:
!conda install -c conda-forge geopy --yes # Installing all required modules
!conda install -c conda-forge folium=0.5.0 --yes 
!pip install requests 
import requests 
!pip install lxml

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



In [34]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd #import Pandas to scrape the web and work with data
pd.options.mode.chained_assignment = None #disable chained warnings for Pandas
pd.set_option('display.max_columns', None) #remove length restrictions
pd.set_option('display.max_rows', None) #remove width restrictions

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans #import k-means from clustering stage

import folium # map rendering library

In [35]:
c_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

from bs4 import BeautifulSoup

soup = BeautifulSoup(c_url, features="lxml")

Table = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', flavor='lxml')

In [36]:
df = Table[0]

In [37]:
print('Row count "as is": ', df.size)

#Remove all rows where Borough is not assigned
zipcodes = df[df.Borough != 'Not assigned']
print('Row count without "Not assigned" neighbourhoods: ', zipcodes.size)

#Copy borough value to neighbourhood if neighbourhood is not assigned
zipcodes.loc[zipcodes.Neighbourhood == 'Not assigned', 'Neighbourhood'] = zipcodes['Borough']

#Grouping the rows by their Postcode
zipcodes = zipcodes.groupby(['Postcode', 'Borough']).agg(lambda x: ', '.join(x))
print('Dataframe grouping complete...')

#Reset index for in the future
zipcodes.reset_index(inplace=True)
print('Index reset complete...')

Row count "as is":  864
Row count without "Not assigned" neighbourhoods:  633
Dataframe grouping complete...
Index reset complete...


In [38]:
zipcodes.shape

(103, 3)

Long and Lat added to table

In [44]:
GEODATA_CSV = 'https://cocl.us/Geospatial_data'

#Toronto coordinates
TORONTO_LAT = 43.70011
TORONTO_LONG = -79.4163

#Foursquare parameters
LIMIT = 100
RADIUS = 500

#Foursquare API URL templates
FS_EXPLORE_URL = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'

In [45]:
#Load data from CSV
geo_data = pd.read_csv(GEODATA_CSV)

#Print some stats
print('Data loaded. Rows: ', geo_data.shape[0])
print('Columns: ', geo_data.columns)


Data loaded. Rows:  103
Columns:  Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')


In [46]:
#Merge frames into new frame using zipcode as a key
geo_zipcodes = pd.merge(
    zipcodes, geo_data,
    how='inner',
    left_on = 'Postcode', right_on = 'Postal Code')

#Drop unwanted 'Postal Code' column
geo_zipcodes.drop('Postal Code', axis=1, inplace=True)

In [50]:
geo_zipcodes

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [52]:
toronto_map = folium.Map(location=[TORONTO_LAT, TORONTO_LONG], zoom_start=11)

#adding markers for the map
for lat, lng, borough, neighbourhood in zip(geo_zipcodes['Latitude'], geo_zipcodes['Longitude'], geo_zipcodes['Borough'], geo_zipcodes['Neighbourhood']):
    label = 'Borough: {}. Neigbourhoods: {}'.format(borough, neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='blue', fill=True, fill_color='#3186cc', fill_opacity=0.7, parse_html=False).add_to(toronto_map)
    
toronto_map